Gambit - giving up material for an advantage in time

In [1]:
import pandas as pd

def iBlend(path_to_ds, file_short_names, sls):

    def tida(sls):
        def read_subm(sls,i):
            tnm = sls["subm"][i]["name"]
            FiN = sls["path"] + tnm + ".csv"
            return pd.read_csv(FiN).rename(columns={'target':tnm, sls["target"]:tnm})
        
        dfs_subm = [read_subm(sls,i) for i in range(len(sls["subm"]))]
        df_subms = pd.merge(dfs_subm[0],  dfs_subm[1], on=['ID'])
        
        for i in range(2, len(sls["subm"])): 
            df_subms = pd.merge(df_subms, dfs_subm[i], on=['ID'])
            
        cols = [col for col in df_subms.columns if col != "ID"]
        short_name_cols = [c.replace(sls["prefix"], '') for c in cols]
        
        weights1,corrects1  = [subm['weight'] for subm in sls["subm"]], [wt for wt in sls["subwts"] ]
        weights2,corrects2  = [subm['weight'] for subm in sls["subm2"]],[wt for wt in sls["subwts2"]]
        weights3,corrects3  = [subm['weight'] for subm in sls["subm3"]],[wt for wt in sls["subwts3"]]
        weights4,corrects4  = [subm['weight'] for subm in sls["subm4"]],[wt for wt in sls["subwts4"]]
        weights5,corrects5  = [subm['weight'] for subm in sls["subm5"]],[wt for wt in sls["subwts5"]]
        
        def alls(x, cs=cols):
            tes = {c: x[c] for c in cs}.items()
            subms_sorted = [
              t[0].replace(sls["prefix"], '')
              for t in sorted(tes,key=lambda k:k[1],reverse=True if sls["sort"]=='desc' else False)]
            return subms_sorted
        
        def correct(x, cs=cols, 
                    w1=weights1, cw1=corrects1, 
                    w2=weights2, cw2=corrects2,
                    w3=weights3, cw3=corrects3,
                    w4=weights4, cw4=corrects4,
                    w5=weights5, cw5=corrects5,
                   ):
            ic = [x['alls'].index(c) for c in short_name_cols]

            mxm = x['abs(mx-m)']

            if   0.00 < mxm <= 0.50:
                cS = [x[cols[j]] * (w1[j] + cw1[ic[j]]) for j in range(len(cols))]
            elif 0.50 < mxm <= 1.00:
                cS = [x[cols[j]] * (w2[j] + cw2[ic[j]]) for j in range(len(cols))]
            elif 1.00 < mxm <= 1.50:
                cS = [x[cols[j]] * (w3[j] + cw3[ic[j]]) for j in range(len(cols))]
            elif 1.50 < mxm <= 2.00:
                cS = [x[cols[j]] * (w4[j] + cw4[ic[j]]) for j in range(len(cols))]
            else:
                cS = [x[cols[j]] * (w5[j] + cw5[ic[j]]) for j in range(len(cols))]
            return sum(cS)

        def amxm(x, cs=cols):
            list_values = x[cs].to_list()
            mxm = abs(max(list_values)-min(list_values))
            return mxm

        df_subms['abs(mx-m)']   = df_subms.apply(lambda x: amxm   (x), axis=1)
        
        df_subms['alls']        = df_subms.apply(lambda x: alls   (x), axis=1)
        df_subms[sls["target"]] = df_subms.apply(lambda x: correct(x), axis=1)
        
        schema_rename = { old_nc:new_shnc for old_nc, new_shnc in zip(cols, short_name_cols) }
        
        df_subms = df_subms.rename(columns=schema_rename)
        df_subms = df_subms.rename(columns={sls["target"]:"ensemble"})
        
        df_subms.insert(loc=1, column=' _ ', value=['   '] * sls["q_rows"])
        
        df_subms[' _ '] = df_subms[' _ '].astype(str)
        pd.set_option('display.max_rows',100)
        pd.set_option('display.float_format', '{:.4f}'.format)
        vcols = ['ID'] + [' _ '] + short_name_cols + [' _ '] + ['abs(mx-m)'] + [' _ '] + ['alls'] + [' _ '] + ['ensemble']
        df_subms = df_subms[vcols]
        display(df_subms.head(8))
        pd.set_option('display.float_format', '{:.7f}'.format)
        df_subms = df_subms.rename(columns={"ensemble":sls["target"]})
        return df_subms

    sample_subm = pd.read_csv(path_to_ds + file_short_names[1] + ".csv")

    def ensemble_tida(sls,submission=sample_subm):   
        sls['sort'] = 'desc'
        dfs = tida(sls)
        dfD = dfs[['ID', sls['target']]]
        dfD.to_csv(f'tida_desc.csv', index=False)
        sls['sort'] = 'asc'
        dfs = tida(sls)
        dfA = dfs[['ID', sls['target']]]
        dfA.to_csv(f'tida_asc.csv',  index=False)
        target,d,a = sls['target'],sls['desc'],sls['asc']
        submission[target] = dfD[target] * d + a * dfA[target]
        return submission

    submission = ensemble_tida(sls)
    
    return submission
    
path_to_ds ='/kaggle/input/21-juli-2025-drw/submission '

file_short_names = ['0.95109','0.95004', '0.95002', '0.94857', '0.90222']

params_14 = {
      'path'   : path_to_ds,                                 
      'sort'   : "dynamic",
      'target' : "prediction",
      'q_rows' : 538_150,
      'prefix' : "subm_",
      'desc'   : 0.32,
      'asc'    : 0.68,
      'subwts' : [+0.015, +0.002, -0.002, -0.005, -0.010],       
      'subm'   : [
         { 'name':file_short_names[0],'weight':0.86, },
         { 'name':file_short_names[1],'weight':0.04, },
         { 'name':file_short_names[2],'weight':0.04, },
         { 'name':file_short_names[3],'weight':0.05, },
         { 'name':file_short_names[4],'weight':0.01, },
      ],
      'subwts2' : [+0.020, +0.002, -0.002, -0.007, -0.013],      
      'subm2'   : [
         { 'name':file_short_names[0],'weight':0.83, },
         { 'name':file_short_names[1],'weight':0.053,},
         { 'name':file_short_names[2],'weight':0.053,},
         { 'name':file_short_names[3],'weight':0.054,},
         { 'name':file_short_names[4],'weight':0.010,},
      ],
      'subwts3' : [+0.025, +0.002, -0.002, -0.010, -0.015],     
      'subm3'   : [
         { 'name':file_short_names[0],'weight':0.82, },
         { 'name':file_short_names[1],'weight':0.057,},
         { 'name':file_short_names[2],'weight':0.057,},
         { 'name':file_short_names[3],'weight':0.057,},
         { 'name':file_short_names[4],'weight':0.010,},
      ],
      'subwts4' : [+0.030, +0.002, -0.002, -0.010, -0.020],    
      'subm4'   : [
         { 'name':file_short_names[0],'weight':0.79, },
         { 'name':file_short_names[1],'weight':0.07, },
         { 'name':file_short_names[2],'weight':0.07, },
         { 'name':file_short_names[3],'weight':0.07, },
         { 'name':file_short_names[4],'weight':0.00, },
      ],
      'subwts5' : [+0.035, +0.002, -0.002, -0.012, -0.023],      
      'subm5'   : [
         { 'name':file_short_names[0],'weight':0.78, },
         { 'name':file_short_names[1],'weight':0.07, },
         { 'name':file_short_names[2],'weight':0.07, },
         { 'name':file_short_names[3],'weight':0.07, },
         { 'name':file_short_names[4],'weight':0.01, },
      ],
    }

params = params_14

df = iBlend ( path_to_ds, file_short_names, params )
df.to_csv('submission.csv', index=False)
display(df)

,ID,_,0.95109,0.95004,0.95002,0.94857,0.90222,_,abs(mx-m),_,alls,_,ensemble
0,1,,-0.0546,-0.1592,-0.1595,-0.1550,-0.1875,,0.1330,,"[0.95109, 0.94857, 0.95004, 0.95002, 0.90222]",,-0.0674
1,2,,0.1370,0.2571,0.2568,0.2605,0.2614,,0.1244,,"[0.90222, 0.94857, 0.95004, 0.95002, 0.95109]",,0.1553
2,3,,-0.6161,-1.4301,-1.4307,-1.4875,-1.0014,,0.8715,,"[0.95109, 0.90222, 0.95004, 0.95002, 0.94857]",,-0.7354
3,4,,-0.0866,-0.2129,-0.2132,-0.2081,-0.2056,,0.1266,,"[0.95109, 0.90222, 0.94857, 0.95004, 0.95002]",,-0.1021
4,5,,0.0594,0.0867,0.0864,0.0953,0.1329,,0.0735,,"[0.90222, 0.94857, 0.95004, 0.95002, 0.95109]",,0.0651
5,6,,-0.2831,-0.6948,-0.6951,-0.6820,-0.8226,,0.5394,,"[0.95109, 0.94857, 0.95004, 0.95002, 0.90222]",,-0.3438
6,7,,0.5151,0.9988,0.9921,1.1359,1.2738,,0.7587,,"[0.90222, 0.94857, 0.95004, 0.95002, 0.95109]",,0.6192
7,8,,-0.0482,-0.1687,-0.1690,-0.2040,-0.1688,,0.1559,,"[0.95109, 0.95004, 0.90222, 0.95002, 0.94857]",,-0.0647


,ID,_,0.95109,0.95004,0.95002,0.94857,0.90222,_,abs(mx-m),_,alls,_,ensemble
0,1,,-0.0546,-0.1592,-0.1595,-0.1550,-0.1875,,0.1330,,"[0.90222, 0.95002, 0.95004, 0.94857, 0.95109]",,-0.0708
1,2,,0.1370,0.2571,0.2568,0.2605,0.2614,,0.1244,,"[0.95109, 0.95002, 0.95004, 0.94857, 0.90222]",,0.1522
2,3,,-0.6161,-1.4301,-1.4307,-1.4875,-1.0014,,0.8715,,"[0.94857, 0.95002, 0.95004, 0.90222, 0.95109]",,-0.7680
3,4,,-0.0866,-0.2129,-0.2132,-0.2081,-0.2056,,0.1266,,"[0.95002, 0.95004, 0.94857, 0.90222, 0.95109]",,-0.1053
4,5,,0.0594,0.0867,0.0864,0.0953,0.1329,,0.0735,,"[0.95109, 0.95002, 0.95004, 0.94857, 0.90222]",,0.0632
5,6,,-0.2831,-0.6948,-0.6951,-0.6820,-0.8226,,0.5394,,"[0.90222, 0.95002, 0.95004, 0.94857, 0.95109]",,-0.3617
6,7,,0.5151,0.9988,0.9921,1.1359,1.2738,,0.7587,,"[0.95109, 0.95002, 0.95004, 0.94857, 0.90222]",,0.5929
7,8,,-0.0482,-0.1687,-0.1690,-0.2040,-0.1688,,0.1559,,"[0.94857, 0.95002, 0.90222, 0.95004, 0.95109]",,-0.0686


,ID,prediction
0,1,-0.0697070
1,2,0.1531607
2,3,-0.7575931
3,4,-0.1042511
4,5,0.0638167
...,...,...
538145,538146,-0.1482612
538146,538147,0.0917190
538147,538148,-0.4760849
538148,538149,0.4921870
